# Coding Exercises

In [ ]:
import numpy as np
import numpy.linalg as nplin
import math
import sklearn.linear_model
import sklearn.metrics
from scipy.special import expit
from sklearn.decomposition import PCA
import scipy as sp

shuffle training data and split into a training and a test set

In [ ]:
def split_data(X, y, frac=0.3, seed=None):
    if seed is not None:
        n_testset = math.ceil(0.3*X.shape[0])
        n_trainset = X.shape[0] - n_testset
        np.random.seed(seed)
        idx = np.arange(X.shape[0])
        idx_shuffled = np.random.permutation(idx)

        test_idx = idx_shuffled[:n_testset]
        train_idx = idx_shuffled[n_testset:]
        X_test = X[test_idx, : ]
        y_test = y[test_idx]
        X_train = X[train_idx, : ]
        y_train = y[train_idx]
        
        return X_test, y_test, X_train, y_train

manual ridge regression

In [ ]:
def ridge_regression(X_test, X_train, y_train, alpha):
    
    Xtr_design =  np.c_[np.ones(X_train.shape[0]), X_train]
    X_te_design = np.c_[np.ones(X_test.shape[0]), X_test]

    inv = nplin.inv(Xtr_design.T @ Xtr_design + alpha * np.identity(Xtr_design.shape[1]))
    weights = inv @ Xtr_design.T @ y_train

    y_pred = X_te_design @ weights
    
    return weights, y_pred

n_fold-crossvalidation over the ridge regression parameter alpha

In [ ]:
def ridgeCV(X, y, n_folds, alphas):

    cv_results_mse = np.zeros((n_folds, len(alphas)))
    np.random.seed(seed=2)

    idx = np.repeat(range(n_folds), math.ceil(X.shape[0]/n_folds))
    idx_shuff = np.random.permutation(idx)  
    idx_shuffled = idx_shuff[:X.shape[0]]
    X_new = np.c_[idx_shuffled, X]
    y_new = np.c_[idx_shuffled, y]

    for i in range(n_folds):
        test_data = X_new[X_new[:,0] == i]
        train_data = X_new[X_new[:,0] != i]
        train_y = y_new[y_new[:,0] != i]
        test_y = y_new[y_new[:,0] == i]
        
        for j in range(len(alphas)):
            y_pred = np.empty([test_data.shape[0], len(alphas)])
            y_pred[:,j:j+1] = ridge_regression_sklearn(test_data[:,1:], train_data[:,1:], train_y[:,1:], alphas[j])[1]
            cv_results_mse[i:i+1:,j:j+1] = mean_squared_error(test_y[:,1:], y_pred[:,j])

    return cv_results_mse

manual linear discriminant analysis

In [ ]:
def compute_lda_weights(x, y):
    
    N0 = sum(y)
    N1 = sum(y==False)
    x0 = x[y==True]
    x1 = x[y==False]
    
    m0 = (1/N0) * np.sum(x0, axis=0)
    m1 = (1/N1) * np.sum(x1, axis=0)
    d0 = np.array(x0-m0)
    d1 = np.array(x1-m1)

    S_w = (d0.T@d0) + (d1.T@d1)
    mdiff = (m1-m0)
    w_lda = nplin.inv(S_w) @ mdiff
    
    return m0, m1, mdiff, w_lda   

manual logistic regression

In [ ]:
def irls_al(X,t):
    t = np.asmatrix(t)
    Phi = np.c_[np.ones(X.shape[0]), X]
    w = np.full((Phi.shape[1], 1), 0.000000001)
    y = expit(Phi @ w)
    acc = np.sum(y==t)/X.shape[0]
    t_label = np.zeros(shape = X.shape[0])
    
    while acc<0.8:
        d = y-(1-y)
        d = np.array(d[:,0]).flatten()
        R = np.diag(d)
        w = w - nplin.inv(Phi.T @ R @ Phi) @ Phi.T @ (y-t.T)
        y = expit(Phi @ w)
        
        t_label[np.array(y[:,0]>0.5).flatten()] = 1
        t_label[np.array(y[:,0]<0.5).flatten()] = 0
        acc = np.sum(t_label==np.array(t).flatten())/X.shape[0]
        print(acc)
        
    return w

manual Support Vector Machines

In [ ]:
def fit_svm(X, t, kernel, C=1.0):

    t = np.array([-1 if l == 0 else 1 for l in t])
    n_samples = len(X)
    K = kernel(X,X)      
    P = np.outer(t,t)*K
    q = np.ones(n_samples)*-1
    A = np.asmatrix(t*1.0)
    b = np.asmatrix(0.0)
    G = np.vstack([np.eye(n_samples),np.eye(n_samples)*-1])
    h = np.hstack([C*np.ones(n_samples),np.zeros(n_samples)])

    assert P.shape == (len(X), len(X))
    assert len(q) == len(X)
    assert A.shape == (1, n_samples) and A.dtype == 'float'
    assert b.shape == (1, 1)
    assert len(G) == 2 * len(X)
    assert len(h) == 2 * len(X)

    return solve_quadratic_program(P, q, A, b, G, h)

def solve_quadratic_program(P, q, A, b, G, h):
    P, q, A, b, G, h = [cvxopt.matrix(var) for var in [P, q, A, b, G, h]]
    minimization = cvxopt.solvers.qp(P, q, G, h, A, b)
    lagr_mult = np.ravel(minimization['x'])
    return lagr_mult


def extract_parameters(X, t, kernel, lagr_mult, threshold=1e-7):
    
    t = np.array([-1 if l == 0 else 1 for l in t])
    K = kernel(X,X)
    
    svs = (lagr_mult>threshold)*(lagr_mult<1.0)
    sv_labels = t[svs]
    intercept = np.mean(sv_labels - lagr_mult[svs]*sv_labels*K[svs,svs])

    return lagr_mult, svs, sv_labels, intercept

In [ ]:
alphas = fit_svm(X_train, t_train, rbf_kernel)

lagr_mult, svs, sv_labels, intercept = extract_parameters(X_train, t_train, rbf_kernel, alphas, threshold=1e-7)

 manual Principal Component Analysis

In [ ]:
def PCA_manual(data):

    cov = np.cov(data.T)
    eig_val, eig_vec = nplin.eig(cov)
    eig_vec = eig_vec.T
    idx = np.argsort(eig_val)[::-1]
    principal_components = eig_vec[idx].T
    fraction_variance_explained = np.empty((data.shape[1],1))
    
    for i in range(data.shape[1]):
        fraction_variance_explained[i] = eig_val[i]/eig_val.sum()

    return fraction_variance_explained, principal_components

In [ ]:
def select_PCs(variance_explained, principal_components, percent_variance=None):
    
    n = np.sum(variance_explained.cumsum()<percent_variance) 
    principal_components_kept = principal_components[:,0:n]
    variance_explained_kept = variance_explained.cumsum()[n]

    return variance_explained_kept, principal_components_kept